# Dataset Creation

This notebook will process the 
[NBA Database](https://www.kaggle.com/datasets/wyattowalsh/basketball/data) dataset from 
[Kaggle](https://www.kaggle.com/), and construct multiple datasets of sequential NBA result data, 
leading up to a game between 2 teams.

The database must be downloaded and stored in `data/` to continue.

Several training, testing and evaluation datasets will be created - 
they are fully described in `data/README.md`.

In [1]:
import pandas as pd
import sqlite3
from tqdm import tqdm

db = sqlite3.connect("nba.sqlite")

In [2]:
# get all seasons in the sample
season_start = "2009-10-01 00:00:00"
season_type = "Regular Season"
season_query = f"""
SELECT
    DISTINCT season_id 
FROM game 
WHERE 
    game_date > '{season_start}' 
    AND season_type = '{season_type}';
"""

season_ids_df = pd.read_sql_query(sql=season_query, con=db)
season_ids = tuple(season_ids_df.season_id.values)
print(season_ids)

('22009', '22010', '22011', '22013', '22014', '22015', '22016', '22017', '22018', '22019', '22020', '22021', '22022')


In [5]:
# combine data into a single table
create_full_game_data = f"""
CREATE TABLE IF NOT EXISTS full_game_data AS
SELECT *
FROM game 
    INNER JOIN line_score USING (game_id)
    INNER JOIN other_stats USING (game_id)
WHERE 
    season_id IN {season_ids};
"""

with db:
    _ = db.execute(create_full_game_data)

In [8]:
# remove table if re-running
with db:
    _ = db.execute("DROP TABLE per_team_game_data")

In [9]:
# create table containing all game data per team
create_per_team_game_data = """
CREATE TABLE IF NOT EXISTS per_team_game_data AS

SELECT 
    season_id AS season_id,
    game_id,
    
    -- team info --
    team_id_home AS team_id,
    game_date_est AS date,
    CASE WHEN wl_home = 'W' THEN 1 ELSE 0 END AS win,
    1 AS home,
    CASE WHEN ABS(pts_home - pts_away) <= 8 THEN 1 ELSE 0 END AS close_game,
    CASE
        WHEN COALESCE(pts_ot10_home, 0) != 0 THEN 10
        WHEN COALESCE(pts_ot9_home, 0) != 0 THEN 9
        WHEN COALESCE(pts_ot8_home, 0) != 0 THEN 8
        WHEN COALESCE(pts_ot7_home, 0) != 0 THEN 7
        WHEN COALESCE(pts_ot6_home, 0) != 0 THEN 6
        WHEN COALESCE(pts_ot5_home, 0) != 0 THEN 5
        WHEN COALESCE(pts_ot4_home, 0) != 0 THEN 4
        WHEN COALESCE(pts_ot3_home, 0) != 0 THEN 3
        WHEN COALESCE(pts_ot2_home, 0) != 0 THEN 2
        WHEN COALESCE(pts_ot1_home, 0) != 0 THEN 1
    ELSE 0 END AS ot_count,
    team_wins_losses_home AS wins_losses,

    -- line score
    pts_qtr1_home AS q1_points,
    pts_qtr2_home AS q2_points,
    pts_qtr3_home AS q3_points,
    pts_qtr4_home AS q4_points,
    COALESCE(pts_ot1_home, 0) + COALESCE(pts_ot2_home, 0) + COALESCE(pts_ot3_home, 0) + COALESCE(pts_ot4_home, 0) + COALESCE(pts_ot5_home, 0) + COALESCE(pts_ot6_home, 0) + COALESCE(pts_ot7_home, 0) + COALESCE(pts_ot8_home, 0) + COALESCE(pts_ot9_home, 0) + COALESCE(pts_ot10_home, 0) AS ot_points,
    pts_home AS final_points,

    -- base stats --
    fgm_home AS field_made,
    fg_pct_home AS field_percent,
    fg3m_home AS three_made,
    fg3_pct_home AS three_percent,
    ftm_home AS free_made,
    ft_pct_home AS free_percent,
    oreb_home AS offensive_rebounds,
    dreb_home AS defensive_rebounds,
    reb_home AS total_rebounds,
    ast_home AS assists,
    stl_home AS steals,
    blk_home AS blocks,
    tov_home AS turnovers,
    pf_home AS fouls,
    plus_minus_home AS plus_minus,

    -- other stats --
    pts_paint_home AS pts_paint,
    pts_2nd_chance_home AS pts_2nd_chance,
    pts_fb_home AS pts_fast_break,
    largest_lead_home AS largest_lead

FROM full_game_data

UNION

SELECT 
    season_id AS season_id,
    game_id,
    
    -- team info --
    team_id_away AS team_id,
    game_date_est AS date,
    CASE WHEN wl_home = 'W' THEN 0 ELSE 1 END AS win,
    0 AS home,
    CASE WHEN ABS(pts_home - pts_away) <= 8 THEN 1 ELSE 0 END AS close_game,
    CASE
        WHEN COALESCE(pts_ot10_away, 0) != 0 THEN 10
        WHEN COALESCE(pts_ot9_away, 0) != 0 THEN 9
        WHEN COALESCE(pts_ot8_away, 0) != 0 THEN 8
        WHEN COALESCE(pts_ot7_away, 0) != 0 THEN 7
        WHEN COALESCE(pts_ot6_away, 0) != 0 THEN 6
        WHEN COALESCE(pts_ot5_away, 0) != 0 THEN 5
        WHEN COALESCE(pts_ot4_away, 0) != 0 THEN 4
        WHEN COALESCE(pts_ot3_away, 0) != 0 THEN 3
        WHEN COALESCE(pts_ot2_away, 0) != 0 THEN 2
        WHEN COALESCE(pts_ot1_away, 0) != 0 THEN 1
    ELSE 0 END AS ot_count,
    team_wins_losses_away AS wins_losses,

    -- line score
    pts_qtr1_away AS q1_points,
    pts_qtr2_away AS q2_points,
    pts_qtr3_away AS q3_points,
    pts_qtr4_away AS q4_points,
    COALESCE(pts_ot1_away, 0) + COALESCE(pts_ot2_away, 0) + COALESCE(pts_ot3_away, 0) + COALESCE(pts_ot4_away, 0) + COALESCE(pts_ot5_away, 0) + COALESCE(pts_ot6_away, 0) + COALESCE(pts_ot7_away, 0) + COALESCE(pts_ot8_away, 0) + COALESCE(pts_ot9_away, 0) + COALESCE(pts_ot10_away, 0) AS ot_points,
    pts_away AS final_points,
    
    -- base stats --
    fgm_away AS field_made,
    fg_pct_away AS field_percent,
    fg3m_away AS three_made,
    fg3_pct_away AS three_percent,
    ftm_away AS free_made,
    ft_pct_away AS free_percent,
    oreb_away AS offensive_rebounds,
    dreb_away AS defensive_rebounds,
    reb_away AS total_rebounds,
    ast_away AS assists,
    stl_away AS steals,
    blk_away AS blocks,
    tov_away AS turnovers,
    pf_away AS fouls,
    plus_minus_away AS plus_minus,

    -- other stats --
    pts_paint_away AS pts_paint,
    pts_2nd_chance_away AS pts_2nd_chance,
    pts_fb_away AS pts_fast_break,
    largest_lead_away AS largest_lead

FROM full_game_data;
"""

with db:
    _ = db.execute(create_per_team_game_data)

In [10]:
# load per-team game data into a dataframe
team_data_df = pd.read_sql_query(
    sql="SELECT * FROM per_team_game_data;",
    con=db,
)

# check size and columns
print(team_data_df.shape)
print(team_data_df.columns.values)

(26248, 34)
['season_id' 'game_id' 'team_id' 'date' 'win' 'home' 'close_game'
 'ot_count' 'wins_losses' 'q1_points' 'q2_points' 'q3_points' 'q4_points'
 'ot_points' 'final_points' 'field_made' 'field_percent' 'three_made'
 'three_percent' 'free_made' 'free_percent' 'offensive_rebounds'
 'defensive_rebounds' 'total_rebounds' 'assists' 'steals' 'blocks'
 'turnovers' 'fouls' 'plus_minus' 'pts_paint' 'pts_2nd_chance'
 'pts_fast_break' 'largest_lead']


In [11]:
# calculate games played in season, and win percentage
games_played = []
win_percent = []

for _, row in team_data_df.iterrows():
    wins, losses = (int(n) for n in row["wins_losses"].split("-"))
    games_played.append(wins + losses)
    win_percent.append(wins / (wins + losses))

# update the dataframe
team_data_df["games_played"] = games_played
team_data_df["win_percent"] = win_percent
team_data_df = team_data_df.drop(columns=["wins_losses"])

# check size and columns
print(team_data_df.shape)
print(team_data_df.columns.values)

(26248, 35)
['season_id' 'game_id' 'team_id' 'date' 'win' 'home' 'close_game'
 'ot_count' 'q1_points' 'q2_points' 'q3_points' 'q4_points' 'ot_points'
 'final_points' 'field_made' 'field_percent' 'three_made' 'three_percent'
 'free_made' 'free_percent' 'offensive_rebounds' 'defensive_rebounds'
 'total_rebounds' 'assists' 'steals' 'blocks' 'turnovers' 'fouls'
 'plus_minus' 'pts_paint' 'pts_2nd_chance' 'pts_fast_break' 'largest_lead'
 'games_played' 'win_percent']


In [12]:
# get list of games to predict
list_games_query = f"""
SELECT
    season_id,
    game_id,
    game_date,
    team_id_home,
    team_id_away,
    matchup_home AS home_vs_away,
    CASE WHEN wl_home = 'W' THEN 1 ELSE 0 END AS home_win,
    pts_home AS points_home,
    pts_away AS points_away
FROM game
WHERE 
    season_id IN {season_ids};
"""

# load games into a dataframe
games_df = pd.read_sql_query(sql=list_games_query, con=db)

# check size and columns
print(games_df.shape)
print(games_df.columns.values)

(15429, 9)
['season_id' 'game_id' 'game_date' 'team_id_home' 'team_id_away'
 'home_vs_away' 'home_win' 'points_home' 'points_away']


In [13]:
# convert dates
team_data_df["date"] = pd.to_datetime(team_data_df["date"])
games_df["game_date"] = pd.to_datetime(games_df["game_date"])

# ensure all numeric columns have the float type
tensor_columns = [
    "win",
    "home",
    "close_game",
    "ot_count",
    "q1_points",
    "q2_points",
    "q3_points",
    "q4_points",
    "ot_points",
    "final_points",
    "field_made",
    "three_made",
    "free_made",
    "offensive_rebounds",
    "defensive_rebounds",
    "total_rebounds",
    "assists",
    "steals",
    "blocks",
    "turnovers",
    "fouls",
    "plus_minus",
    "pts_paint",
    "pts_2nd_chance",
    "pts_fast_break",
    "largest_lead",
    "games_played",
    "field_percent",
    "three_percent",
    "free_percent",
    "win_percent",
]

for column in tensor_columns:
    team_data_df[column] = team_data_df[column].astype(float)

In [14]:
# split training and evaluation data halfway through the 2021-22 season (all-star weekend)
split_on_date = pd.Timestamp("2022-02-21")
print(f"evaluation data from date: {split_on_date}")

# save final week of 2022-23 season data as an example
final_week_date = pd.Timestamp("2023-04-04")
print(f"final week example data from date: {final_week_date}")

evaluation data from date: 2022-02-21 00:00:00
final week example data from date: 2023-04-04 00:00:00


In [15]:
# utility function to work out current streak from list of results
def current_streak(game_results: list[int], target: int) -> int:
    if game_results[0] != target:
        return 0

    streak = 1
    for g in game_results[1:]:
        if g != target:
            break
        streak += 1
    return streak

In [16]:
# lists to track the various datasets
training_data = []
evaluation_data_half = []
evaluation_data_full = []
final_week_data = []
training_streaks_short = []
training_streaks_long = []
evaluation_streaks_short = []
evaluation_streaks_long = []


# loop through each game, constructing the sample data from the team data
for idx, game in tqdm(games_df.iterrows()):
    home_team_games_df = (
        team_data_df.loc[
            (team_data_df["season_id"] == game["season_id"])
            & (team_data_df["team_id"] == game["team_id_home"])
            & (team_data_df["date"] < game["game_date"])
        ]
        .sort_values(by="date")
        .tail(10)
    )

    if len(home_team_games_df) < 10:
        # skip if not enough previous data
        continue

    away_team_games_df = (
        team_data_df.loc[
            (team_data_df["season_id"] == game["season_id"])
            & (team_data_df["team_id"] == game["team_id_away"])
            & (team_data_df["date"] < game["game_date"])
        ]
        .sort_values(by="date")
        .tail(10)
    )

    if len(away_team_games_df) < 10:
        # skip if not enough previous data
        continue

    home_team_games = []
    home_team_opps = []
    for _, row in home_team_games_df.iterrows():
        home_team_games.append(row.to_list()[4:])
        home_team_opps.append(
            team_data_df.loc[
                (team_data_df["game_id"] == row["game_id"])
                & (team_data_df["team_id"] != game["team_id_home"])
            ]
            .iloc[0]
            .to_list()[8:]
        )

    away_team_games = []
    away_team_opps = []
    for _, row in away_team_games_df.iterrows():
        away_team_games.append(row.to_list()[4:])
        away_team_opps.append(
            team_data_df.loc[
                (team_data_df["game_id"] == row["game_id"])
                & (team_data_df["team_id"] != game["team_id_away"])
            ]
            .iloc[0]
            .to_list()[8:]
        )

    combined = []
    for a, b, c, d in zip(
        home_team_games,
        home_team_opps,
        away_team_games,
        away_team_opps,
    ):
        combined.append(a + b + c + d)

    game["game_date"] = game["game_date"].strftime("%Y-%m-%d")  # make date serializable
    _game_dict = {
        "info": game.to_dict(),
        "data": combined,
    }

    home_results = [h[0] for h in home_team_games]
    home_results.reverse()
    away_results = [a[0] for a in away_team_games]
    away_results.reverse()
    _streaks = {
        current_streak(home_results, 1 - game["home_win"]),
        current_streak(away_results, game["home_win"]),
    }

    if pd.Timestamp(game["game_date"]) < split_on_date:
        training_data.append(_game_dict)

        if any([s > 5 for s in _streaks]):
            training_streaks_long.append(_game_dict)
        if {3, 4} & _streaks:
            training_streaks_short.append(_game_dict)

    else:
        if game["season_id"] == "22021":
            evaluation_data_half.append(_game_dict)
        elif game["season_id"] == "22022":
            evaluation_data_full.append(_game_dict)

        if any([s > 5 for s in _streaks]):
            evaluation_streaks_long.append(_game_dict)
        if {3, 4} & _streaks:
            evaluation_streaks_short.append(_game_dict)

    if pd.Timestamp(game["game_date"]) > final_week_date:
        final_week_data.append(_game_dict)

0it [00:00, ?it/s]

15429it [31:33,  8.15it/s] 


In [17]:
# create final few collections of datasets
evaluation_data = evaluation_data_full + evaluation_data_half
complete_data = evaluation_data + training_data
all_streaks = (
    training_streaks_long
    + training_streaks_short
    + evaluation_streaks_long
    + evaluation_streaks_long
)

print(
    f"Have {len(training_data)} training samples, "
    f"{len(evaluation_data)} evaluation samples, "
    f"{len(final_week_data)} examples, "
    f"and {len(all_streaks)} streaks."
)

Have 11581 training samples, 1398 evaluation samples, 42 examples, and 2600 streaks.


In [18]:
# check and save to file
for _records, _name in [
    (training_data, "training_df"),
    (evaluation_data, "evaluation_df_all"),
    (evaluation_data_half, "evaluation_df_half_21_22"),
    (evaluation_data_full, "evaluation_df_full_22_23"),
    (complete_data, "complete_df"),
    (final_week_data, "final_week_df"),
    (training_streaks_short, "training_streaks_short_df"),
    (training_streaks_long, "training_streaks_long_df"),
    (evaluation_streaks_short, "evaluation_streaks_short_df"),
    (evaluation_streaks_long, "evaluation_streaks_long_df"),
]:
    _df = pd.DataFrame.from_records(_records)
    _file = f"parquet/{_name}.parquet"
    _df.to_parquet(_file)
    print(f"Saved df {_name} with shape {_df.shape} to {_file}")

Saved df training_df with shape (11581, 2) to parquet/training_df.parquet
Saved df evaluation_df_all with shape (1398, 2) to parquet/evaluation_df_all.parquet
Saved df evaluation_df_half_21_22 with shape (347, 2) to parquet/evaluation_df_half_21_22.parquet
Saved df evaluation_df_full_22_23 with shape (1051, 2) to parquet/evaluation_df_full_22_23.parquet
Saved df complete_df with shape (12979, 2) to parquet/complete_df.parquet
Saved df final_week_df with shape (42, 2) to parquet/final_week_df.parquet
Saved df training_streaks_short_df with shape (1845, 2) to parquet/training_streaks_short_df.parquet
Saved df training_streaks_long_df with shape (623, 2) to parquet/training_streaks_long_df.parquet
Saved df evaluation_streaks_short_df with shape (239, 2) to parquet/evaluation_streaks_short_df.parquet
Saved df evaluation_streaks_long_df with shape (66, 2) to parquet/evaluation_streaks_long_df.parquet
